<a href="https://colab.research.google.com/github/gathungugabriel/WEB-CRAWLING-CRAIGSLIST-JOB-SCRAPER/blob/main/WEB_CRAWLING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,343 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,742 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/ma

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

def scrape_craigslist_geo():
    url = 'https://geo.craigslist.org/iso/us'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = []
        for li_tag in soup.find_all('li'):
            a_tag = li_tag.find('a', href=True)
            if a_tag and a_tag['href'].endswith('craigslist.org'):
                links.append(a_tag['href'])
        return links
    else:
        print(f"Failed to fetch {url}. Status code: {response.status_code}")
        return None

def scrape_craigslist_links(links):
    all_links = []
    for link in links:
        url = f"{link}/search/crg?postedToday=1#search=1~thumb~0~0"
        print(f"Fetching data from: {url}")
        try:
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')
            driver = webdriver.Chrome(options=options)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            # Find all <a> elements with class 'cl-app-anchor' within the desired structure
            links = soup.select('.result-info .title-blob a')
            for link in links:
                label = link.find('span', class_='label')
                if label:
                    href = link['href']
                    all_links.append(href)
            driver.quit()
        except Exception as e:
            print(f"An error occurred while fetching {url}: {e}")
    return all_links

def scrape_posting_data(links):
    posting_data = []
    for link in links:
        print(f"Fetching data from: {link}")
        try:
            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')
            driver = webdriver.Chrome(options=options)
            driver.get(link)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            job_title_tag = soup.find('span', id='titletextonly')
            job_description_tag = soup.find('section', id='postingbody')
            if job_title_tag and job_description_tag:
                job_title = job_title_tag.text.strip()
                job_description = job_description_tag.text.strip()
                posting_data.append({'Job Title': job_title, 'Job Description': job_description, 'Job Link': link})
            driver.quit()
        except Exception as e:
            print(f"An error occurred while fetching {link}: {e}")
    return posting_data

# Call the function to scrape Craigslist geo locations
craigslist_geo_links = scrape_craigslist_geo()

# Call the function to scrape links from Craigslist for each geo location
all_links = scrape_craigslist_links(craigslist_geo_links)

# Call the function to scrape job titles and descriptions from each link
posting_data = scrape_posting_data(all_links)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(posting_data)

# Display the DataFrame
print(df)


Fetching data from: https://abilene.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://akroncanton.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://albanyga.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://albany.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://albuquerque.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://altoona.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://amarillo.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://ames.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://anchorage.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://annapolis.craigslist.org/search/crg?postedToday=1#search=1~thumb~0~0
Fetching data from: https://annarbor.cra

In [5]:
df

,Job Title,Job Description,Job Link
0,Seeking digital artists / designers to join ne...,QR Code Link to This Post\n\n\nSell Your Artwo...,https://pittsburgh.craigslist.org/crg/d/carneg...
1,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://chicago.craigslist.org/nwc/crg/d/chica...
2,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://chicago.craigslist.org/chc/crg/d/chica...
3,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://cincinnati.craigslist.org/crg/d/addyst...
4,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://chicago.craigslist.org/chc/crg/d/chica...
...,...,...,...
230,LOOKING FOR SOCIAL MEDIA ASSISTANT,QR Code Link to This Post\n\n\nHome Services c...,https://losangeles.craigslist.org/lac/crg/d/lo...
231,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://evansville.craigslist.org/crg/d/evansv...
232,High Cholesterol Studies,QR Code Link to This Post\n\n\n High Cholester...,https://cincinnati.craigslist.org/crg/d/addyst...
233,EXPIERENCED Lifeline Field Reps Earn up to $29...,QR Code Link to This Post\n\n\nWe are looking ...,https://indianapolis.craigslist.org/crg/d/indi...


In [6]:
import pandas as pd

# Assuming df is your DataFrame containing the job data
df.to_csv('job_data.csv', index=False)
